In [708]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc,confusion_matrix,    f1_score, precision_score, recall_score,accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import time
from sklearn.preprocessing import OneHotEncoder

np.random.seed(1234)
from sklearn import model_selection


In [709]:
start_time1 = time.time()
heading =['age', 'workclass', 'fnlwgt', 'education', 'education-num','marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']
train_data = pd.read_csv("Census Income Data Set/adult.data", header=None, names = heading,skipinitialspace = True, na_values='?')
test_data = pd.read_csv("Census Income Data Set/adult.test", header=None, names = heading,skipinitialspace = True,skiprows=1 ,na_values='?')


In [710]:
#data preprocessing for test dataset
test_data['income'] = test_data.apply(lambda row: row['income'].replace(".",""), axis=1)
test_data.value_counts('income')


income
<=50K    12435
>50K      3846
dtype: int64

In [711]:
df = train_data
tdf = test_data

In [712]:
# Remove invalid data from table
df = df[(df.astype(str) != '?').all(axis=1)]
tdf = tdf[(tdf.astype(str) != '?').all(axis=1)]

df = pd.get_dummies(df, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex'])
tdf = pd.get_dummies(tdf, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex'])



In [713]:
df['income_bi'] = df.apply(lambda row: 1 if '>50K'in row['income'] else 0, axis=1)# Remove redundant columns
df = df.drop(['income','fnlwgt','capital-gain','capital-loss','native-country'], axis=1)
tdf['income_bi'] = tdf.apply(lambda row: 1 if '>50K'in row['income'] else 0, axis=1)# Remove redundant columns
tdf = tdf.drop(['income','fnlwgt','capital-gain','capital-loss','native-country'], axis=1)

train_x = df.drop(['income_bi'], axis=1).to_numpy()
train_y = df['income_bi'].to_numpy()
test_x = tdf.drop(['income_bi'], axis=1).to_numpy()
test_y = tdf['income_bi'].to_numpy()

In [714]:
# data = np.array(df)
# X = data[:, :-1]
# y = data[:, -1]


In [715]:
# design a decision tree
class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature        
        self.threshold = threshold    
        self.left = left              
        self.right = right            
        self.value = value           

In [716]:
# gini index
def cost_gini(y):
    unique_classes = np.unique(y)
    total_samples = len(y)
    gini = 1.0

    for cls in unique_classes:
        p = np.sum(y == cls) / total_samples
        gini -= p ** 2

    return gini
#cross entropy
def cost_entropy(labels):
    class_probs = np.bincount(labels) / len(labels)
    class_probs = class_probs[class_probs > 0]            
    return -np.sum(class_probs * np.log2(class_probs))  


In [717]:
# 选择最佳分裂特征和阈值
#X 是一个包含特征数据的矩阵，其中每一行代表一个数据样本，每一列代表一个特征。
#y 是一个包含目标变量（类别标签）的向量，与特征矩阵 X 中的样本一一对应。
def find_best_split(X, y, cost_func):
    num_samples, num_features = X.shape
    best_cost = 1.0
    best_feature = None
    best_threshold = None

    for feature_idx in range(num_features):
        thresholds = np.unique(X[:, feature_idx])
        for threshold in thresholds:
            left_mask = X[:, feature_idx] < threshold
            right_mask = X[:, feature_idx] >= threshold

            left_cost = cost_func(y[left_mask])
            right_cost = cost_func(y[right_mask])
        
            weighted_cost = (left_cost * np.sum(left_mask) + right_cost * np.sum(right_mask)) / num_samples
            
            if weighted_cost < best_cost:
                best_cost = weighted_cost
                
                best_feature = feature_idx
                best_threshold = threshold
    return best_feature, best_threshold


In [718]:
def build_tree(X, y, cost_func, depth=0, max_depth=None):
    num_samples, num_features = X.shape   
    unique_classes, counts = np.unique(y, return_counts=True)
    majority_class_idx = np.argmax(counts)

    # 终止条件：节点只包含一个类别或达到最大深度
    if len(unique_classes) == 1 or (max_depth is not None and depth == max_depth):
        return TreeNode(value=unique_classes[majority_class_idx])
    
    # 寻找最佳分裂特征和阈值
    best_feature, best_threshold = find_best_split(X, y, cost_func)

    # 如果无法继续分裂，返回多数类别
    if best_feature is None:
        return TreeNode(value=unique_classes[majority_class_idx])

    # 根据最佳分裂特征和阈值分裂数据
    left_mask = X[:, best_feature] < best_threshold
    right_mask = X[:, best_feature] >= best_threshold

    left_tree = build_tree(X[left_mask], y[left_mask], cost_func, depth + 1, max_depth)
    right_tree = build_tree(X[right_mask], y[right_mask], cost_func, depth + 1, max_depth)

    return TreeNode(feature=best_feature, threshold=best_threshold, left=left_tree, right=right_tree)

In [719]:
# 预测单个样本
def predict_tree(node, x):
    if node.value is not None:
        return node.value
    if x[node.feature] < node.threshold:
        return predict_tree(node.left, x)
    else:
        return predict_tree(node.right, x)

In [720]:
tree = build_tree(train_x, train_y, cost_entropy, max_depth=5, )

In [721]:
y_pred = [predict_tree(tree, x) for x in train_x]

In [722]:
# train_y=train_y.tolist()

In [723]:
#run model on training set
accuracy = accuracy_score(train_y, y_pred)
classification_rep = classification_report(train_y, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.8234083719787476
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.95      0.89     24720
           1       0.74      0.41      0.53      7841

    accuracy                           0.82     32561
   macro avg       0.79      0.68      0.71     32561
weighted avg       0.81      0.82      0.80     32561



In [724]:
#run model on testing set
y_pred = [predict_tree(tree, x) for x in test_x]
accuracy = accuracy_score(test_y, y_pred)
classification_rep = classification_report(test_y, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.8250721700141269
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.95      0.89     12435
           1       0.73      0.41      0.53      3846

    accuracy                           0.83     16281
   macro avg       0.78      0.68      0.71     16281
weighted avg       0.81      0.83      0.81     16281



In [725]:
train_acc = []
test_acc = []
model_choices=[]

#find the best depth
for depth in range(1,10):
    tree = build_tree(train_x, train_y, cost_entropy, max_depth=depth)
    y_pred = [predict_tree(tree, x) for x in train_x]
    train_accuracy = accuracy_score(train_y, y_pred)
    classification_rep = classification_report(train_y, y_pred)

    y_pred = [predict_tree(tree, x) for x in test_x]
    test_accuracy = accuracy_score(test_y, y_pred)
    classification_rep = classification_report(test_y, y_pred)

    model_choices.append(depth)
    test_acc.append(test_accuracy)
    train_acc.append(train_accuracy)
        

acc = {model_choices[i]: test_acc[i] for i in range(len(test_acc))}
acc

/Users/chloezhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/chloezhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/chloezhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Use

{1: 0.7637737239727289,
 2: 0.8219396842945765,
 3: 0.8232909526441865,
 4: 0.8253178551686015,
 5: 0.8250721700141269,
 6: 0.8310300350101345,
 7: 0.8350838400589644,
 8: 0.8349609974817271,
 9: 0.8350224187703458}

In [726]:
best_depth = max(acc, key= lambda x: acc[x])
best_depth

7

In [727]:
best_func = {}
similarity_func = [cost_entropy, cost_gini]
for s in similarity_func:
    tree = build_tree(train_x, train_y, s, max_depth=best_depth)
    y_pred = [predict_tree(tree, x) for x in train_x]
    train_accuracy = accuracy_score(train_y, y_pred)
    classification_rep = classification_report(train_y, y_pred)


    y_pred = [predict_tree(tree, x) for x in test_x]
    test_accuracy = accuracy_score(test_y, y_pred)
    classification_rep = classification_report(test_y, y_pred)
    best_func[s] = test_accuracy

best_similarity = max(best_func, key= lambda x: best_func[x])  
print("best_similarity_function",best_similarity)
best_func

best_similarity_function <function cost_entropy at 0x7fd0cc016820>


{<function __main__.cost_entropy(labels)>: 0.8350838400589644,
 <function __main__.cost_gini(y)>: 0.8347153123272526}